![](http://interactive.blockdiag.com/image?compression=deflate&encoding=base64&src=eJxLyslPzk7JTExXqOZSUFAvLE0tqlQoz8zOLEgFiqpzKejaKRSlFpfmlBSDmOqefsGuQSEKJfkKBfnFJelAKXVrrloAN3wW0g)

# Query Wikipedia

In [ ]:
!pip install requests

In [ ]:
!pip install Wikipedia 

In [ ]:
!pip install psycopg2

In [3]:
import requests
import wikipedia
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import re
from sklearn.neighbors import NearestNeighbors
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor

# Define base_url

In [4]:
base_url = "https://en.wikipedia.org/w/api.php"

In [5]:
base_url

'https://en.wikipedia.org/w/api.php'

# Function

In [6]:
def generate_category(category):
    '''
    format a category for insertion in to a wikipedia api call
    '''
    category = re.sub('\s','+',category)
    return category

In [7]:
def generate_query(category):
    '''
    Format an api call for requests
    '''
    query = """
            http://en.wikipedia.org/w/api.php?
            action=query&explaintext=True&
            format=json&
            list=categorymembers&
            cmtitle=Category:{}& 
            cmlimit=max
            """.format(generate_category(category))
    query = re.sub('\s','',query)
    return query

In [8]:
def execute_category_query(category):
    '''
    Executes a category query and returns a 
    DataFrame of the category members
    '''
    
    r = requests.get(generate_query(category))
    response = r.json()
    
    df = pd.DataFrame(response['query']['categorymembers'])
    df['category'] = category
    
    return df

In [9]:
def get_df_of_categories(df):
    '''
    Splits df datafram into one without category mask and one with the category mask.
    Categories are obtained which are used to fetch pages.
    '''
    category_mask = df['title'].str.contains('Category:')
    pages1_df = df[~category_mask]
    categories_df = df[category_mask]
    categories = categories_df[categories_df['title'].str.contains('Category:')]['title'].str.replace('Category:','').tolist()
    pages_list = []
    if len(categories) > 0:
        for cat in categories:
            pages_list.append(execute_category_query(cat))
    pages2_df = pd.concat(pages_list)
    return pages1_df, pages2_df, categories_df

# Machine Learning Pages

In [10]:
generate_category('machine learning')

'machine+learning'

In [11]:
ml_df = execute_category_query("machine learning")

In [12]:
ml_pages1_df, ml_pages2_df, ml_categories1_df = get_df_of_categories(ml_df)

In [13]:
ml_pages1_df.shape

(200, 4)

In [14]:
ml_pages2_df.shape

(854, 4)

In [15]:
ml_pages2_df, ml_pages3_df, ml_categories2_df = get_df_of_categories(ml_pages2_df)

In [16]:
ml_pages3_df, ml_pages4_df, ml_categories3_df = get_df_of_categories(ml_pages3_df)

#### No more categories found!

In [17]:
ml_pages4_df[ml_pages4_df['title'].str.contains('Category:')]

,ns,pageid,title,category


#### No more categories!!! 

#### Make one df for pages and one for categories

In [18]:
ml_pages = pd.concat([ml_pages1_df, ml_pages2_df, ml_pages3_df, ml_pages4_df])

In [19]:
ml_pages.shape

(1618, 4)

#### Are pageid in ml_categories3_df already in ml_pages dataframe?

In [20]:
# Reset index for the ml_pages
ml_pages = ml_pages.reset_index(drop=True)

In [21]:
for pgid in ml_categories3_df['pageid']:
    if pgid in np.array(ml_pages['pageid']):
        print('pageid found')
    else:
        print(pgid,'pageid not found.  Get the pages again')

49119651 pageid not found.  Get the pages again
3735046 pageid not found.  Get the pages again
15358447 pageid not found.  Get the pages again
37176176 pageid not found.  Get the pages again
15358447 pageid not found.  Get the pages again
37176176 pageid not found.  Get the pages again
1718975 pageid not found.  Get the pages again
30174957 pageid not found.  Get the pages again
22718453 pageid not found.  Get the pages again


In [22]:
#ml_pages4_df, ml_pages5_df, ml_categories4_df = get_df_of_categories(ml_pages4_df)

In [23]:
categories4 = ml_categories3_df[ml_categories3_df['title'].str.contains('Category:')]['title'].str.replace('Category:','').tolist()
pages_list = []
if len(categories4) > 0:
    for cat in categories4:
        pages_list.append(execute_category_query(cat))
ml_pages5_df = pd.concat(pages_list)

### No more categories exist to pull pageids.

In [24]:
ml_pages5_df[ml_pages5_df['title'].str.contains('Category:')]

,ns,pageid,title,category


In [25]:
ml_pages = pd.concat([ml_pages1_df, ml_pages2_df, ml_pages3_df, ml_pages4_df, ml_pages5_df])

In [26]:
ml_pages = ml_pages.drop('ns', 1)

In [27]:
# Reset index for the ml_pages
ml_pages = ml_pages.reset_index(drop=True)

#### Drop duplicates

In [28]:
ml_pages.shape

(1714, 3)

In [29]:
ml_pages = ml_pages.drop_duplicates(subset=['pageid'], keep='first')

In [30]:
ml_pages.shape

(1108, 3)

In [31]:
# Reset index for the ml_pages
ml_pages = ml_pages.reset_index(drop=True)

In [32]:
len(ml_pages['pageid'].unique())

1108

In [133]:
len(ml_pages['category'].unique())

46

In [136]:
ml_pages['category'] = 'machine learning'

In [34]:
ml_pages.head()

,pageid,title,category
0,54972729,User:CustIntelMngt/sandbox/Customer Intelligen...,machine learning
1,43385931,Data exploration,machine learning
2,49082762,List of datasets for machine learning research,machine learning
3,233488,Machine learning,machine learning
4,53587467,Outline of machine learning,machine learning


#### Gather all the categories along with titles

In [35]:
ml_categories_df = pd.concat([ml_categories1_df,ml_categories2_df,ml_categories3_df])

In [36]:
ml_categories_df = ml_categories_df.reset_index(drop=True)

### Drop duplicates in category and also columns 'ns'

In [37]:
ml_categories_df = ml_categories_df.drop('ns', 1)

In [38]:
ml_categories_df = ml_categories_df.drop_duplicates(subset=['pageid'], keep='first')

In [39]:
ml_categories_df.shape

(48, 3)

In [40]:
ml_categories_df = ml_categories_df.reset_index(drop=True)

In [41]:
ml_categories_df['category'].value_counts()

machine learning                             30
Evolutionary algorithms                       3
Markov models                                 2
Cluster analysis                              2
Artificial neural networks                    2
Genetic algorithms                            1
Statistical natural language processing       1
Graphical models                              1
Classification algorithms                     1
Datasets in machine learning                  1
Latent variable models                        1
Structured prediction                         1
Dimension reduction                           1
Data mining and machine learning software     1
Name: category, dtype: int64

In [42]:
ml_categories_df = ml_categories_df.reset_index(drop=True)

In [43]:
ml_categories_df.head()

,pageid,title,category
0,33547387,Category:Applied machine learning,machine learning
1,42936114,Category:Artificial neural networks,machine learning
2,1718975,Category:Bayesian networks,machine learning
3,1991254,Category:Classification algorithms,machine learning
4,22532673,Category:Cluster analysis,machine learning


# Function to fetch the content for pageid/title

In [44]:
def fetch_content(df):
    '''
    Fetches content for the pageids.
    '''
    content_ml = []
    not_found = []
    for pageid in df['pageid']:
        try:
            page = wikipedia.WikipediaPage(pageid = pageid)
            content = page.content
            title = page.title
            content_ml.append([pageid, content])
        except:
            not_found.append(pageid)
    return content_ml

In [45]:
content_ml = fetch_content(ml_pages)

In [46]:
content_ml_df = pd.DataFrame(content_ml, columns= ("pageid","content"))

In [137]:
all_ml_df = content_ml_df.merge(ml_pages, how= "left", on="pageid")

In [138]:
all_ml_df = all_ml_df[["pageid","title","content","category"]]

In [139]:
all_ml_df.shape

(1103, 4)

#### No duplicates

In [140]:
all_ml_df[all_ml_df.duplicated('pageid')].index.values

array([], dtype=int64)

### Write to csv

In [141]:
all_ml_df.to_csv('ml.csv', index=False)

In [142]:
ml_categories_df.to_csv('category.csv', index=False)

# Business Software Pages

In [53]:
generate_category('business software')

'business+software'

In [54]:
bs_df = execute_category_query("business software")

In [55]:
bs_pages1_df, bs_pages2_df, bs_categories1_df = get_df_of_categories(bs_df)

In [56]:
bs_pages2_df, bs_pages3_df, bs_categories2_df = get_df_of_categories(bs_pages2_df)

In [65]:
bs_pages3_df.head()

,category,ns,pageid,title
0,School-administration software,0.0,15549320.0,Automate the Schools
1,School-administration software,0.0,4926702.0,BCeSIS
2,School-administration software,0.0,40247755.0,Education Management Information System
3,School-administration software,0.0,20515903.0,Electronic grade book
4,School-administration software,0.0,24810600.0,Fedena


In [63]:
bs_pages3_df[bs_pages3_df['title'].str.contains('Category:')]

,category,ns,pageid,title


In [72]:
bs_pages3_df.shape

(2371, 4)

In [73]:
bs_pages = pd.concat([bs_pages1_df, bs_pages2_df, bs_pages3_df])

In [74]:
bs_pages = bs_pages.drop('ns', 1)

In [75]:
bs_pages = bs_pages.drop_duplicates(subset=['pageid'], keep='first')

In [76]:
bs_pages['pageid'] = bs_pages['pageid'].astype(int)

In [77]:
bs_pages = bs_pages.reset_index(drop=True)

In [79]:
bs_pages.shape

(3052, 3)

In [80]:
bs_pages = bs_pages[['pageid', 'title', 'category']]

In [134]:
bs_pages['category'] = 'business software'

In [135]:
bs_pages.head()

,pageid,title,category
0,1037763,Business software,business software
1,41270069,AccuSystems,business software
2,5211212,Active policy management,business software
3,28502793,Alexandria (library software),business software
4,44133735,Alteryx,business software


In [73]:
#pdbs_drop = bs_pages[bs_pages.duplicated('pageid')].index.values 

In [74]:
#indexes_to_keep = set(range(bs_pages.shape[0])) - set(pdbs_drop)
#bs_pages_sliced = bs_pages.take(list(indexes_to_keep))

In [75]:
#bs_pages = bs_pages_sliced

In [82]:
bs_categories_df = pd.concat([bs_categories1_df,bs_categories2_df])

In [83]:
bs_categories_df.shape

(121, 4)

In [84]:
bs_categories_df = bs_categories_df.drop('ns', 1)

In [85]:
bs_categories_df = bs_categories_df.drop_duplicates(subset=['pageid'], keep='first')

In [86]:
bs_categories_df = bs_categories_df.reset_index(drop=True)

In [87]:
bs_categories_df = bs_categories_df[['pageid', 'title', 'category']]

In [88]:
bs_categories_df['pageid'] = bs_categories_df['pageid'].astype(int)

In [84]:
#bs_categories_df = bs_categories_df.reset_index(drop=True)

In [85]:
#bs_cat_indexes_to_keep = set(range(bs_categories_df.shape[0])) - set(bspgd)
#bs_categories_df_sliced = bs_categories_df.take(list(indexes_to_keep))

In [89]:
bs_categories_df = bs_categories_df.reset_index(drop=True)

In [87]:
#bs_categories_df['category'] = 'business software'

In [90]:
categories_df = pd.concat([ml_categories_df, bs_categories_df])

In [91]:
categories_df.to_csv('category.csv', index=False)

In [132]:
len(categories_df['category'].unique())

35

### Fetch Business Software pages

In [93]:
content1_bs = fetch_content(bs_pages1_df)

In [94]:
content1_bs_df = pd.DataFrame(content1_bs, columns= ("pageid","content"))

In [107]:
content1_bs_df.shape

(295, 2)

In [96]:
content2_bs = fetch_content(bs_pages2_df)

In [97]:
content2_bs_df = pd.DataFrame(content2_bs, columns= ("pageid","content"))

In [108]:
content2_bs_df.shape

(1456, 2)

In [99]:
content3_bs_df = fetch_content(bs_pages3_df)

In [100]:
content3_bs_df = pd.DataFrame(content3_bs_df, columns= ("pageid","content"))

In [109]:
content3_bs_df.shape

(2356, 2)

In [106]:
content_bs_df = pd.concat([content1_bs_df, content2_bs_df, content3_bs_df])

In [143]:
all_bs_df = content_bs_df.merge(bs_pages, how= "left", on="pageid")

In [144]:
all_bs_df = all_bs_df[["pageid","title","category","content"]]

In [145]:
all_bs_df.shape

(4107, 4)

In [146]:
all_ml_df.shape

(1103, 4)

In [ ]:
#all_bs_df['category'] = 'business software'

In [147]:
all_pages_df = pd.concat([all_ml_df, all_bs_df])

In [148]:
len(all_pages_df['category'].unique())

2

In [149]:
all_pages_df = all_pages_df[["pageid","title","category","content"]]

In [150]:
all_pages_df.head()

,pageid,title,category,content
0,54972729.0,User:CustIntelMngt/sandbox/Customer Intelligen...,machine learning,\n= Customer Intelligence Management =\n\n\n==...
1,43385931.0,Data exploration,machine learning,Data exploration is an approach similar to ini...
2,49082762.0,List of datasets for machine learning research,machine learning,These datasets are used for machine learning r...
3,233488.0,Machine learning,machine learning,Machine learning is the subfield of computer s...
4,53587467.0,Outline of machine learning,machine learning,The following outline is provided as an overvi...


#### Write to csv

In [151]:
all_bs_df.to_csv('bs.csv', index=False)

In [152]:
bs_categories_df.to_csv('bs_category.csv', index=False)

In [153]:
all_pages_df.to_csv('allpages.csv', index=False)

In [ ]:
import csv
#test_csv_df = pd.read_csv('../prj4/semantic_search/ml_csv')

# DB setup SQLAlchemy connection

 Postgres DB & Tables
 
Create tables

CREATE TABLE category1_tbl (categoryid INTEGER PRIMARY KEY, title TEXT, category TEXT);

CREATE TABLE page1_tbl (pageid INTEGER PRIMARY KEY, title TEXT, content TEXT, category TEXT);

CREATE TABLE page1_category1_tbl (pageid_categoryid INTEGER, pageid INTEGER REFERENCES page_tbl(pageid), categoryid INTEGER REFERENCES category_tbl(categoryid));

In [154]:
from sqlalchemy import create_engine

connection = create_engine("postgresql://postgres@postgres/postgres")

In [162]:
##  read operations to local database
sql = """
SELECT count(*) FROM category1_tbl
"""
pd.read_sql(sql, con=connection)

,count
0,157


In [163]:
## Checking page_Tbl
sql = """
SELECT count(*) FROM page1_tbl
"""
pd.read_sql(sql, con=connection)

,count
0,5210


In [157]:
page_df=pd.read_sql_query('select * from page1_tbl',con=connection)

In [165]:
page_df.head()

,pageid,title,category,content
0,54972729.0,User:CustIntelMngt/sandbox/Customer Intelligen...,machine learning,\n= Customer Intelligence Management =\n\n\n==...
1,43385931.0,Data exploration,machine learning,Data exploration is an approach similar to ini...
2,49082762.0,List of datasets for machine learning research,machine learning,These datasets are used for machine learning r...
3,233488.0,Machine learning,machine learning,Machine learning is the subfield of computer s...
4,53587467.0,Outline of machine learning,machine learning,The following outline is provided as an overvi...


## Write to postgres using SQLAlchemy and Pandas
> Check status on server: <br>
>`SELECT * FROM pg_stat_activity;`

In [159]:
all_pages_df.to_sql("page1_tbl", con=connection, index=False, if_exists="replace")

In [160]:
categories_df.head()

,pageid,title,category
0,33547387,Category:Applied machine learning,machine learning
1,42936114,Category:Artificial neural networks,machine learning
2,1718975,Category:Bayesian networks,machine learning
3,1991254,Category:Classification algorithms,machine learning
4,22532673,Category:Cluster analysis,machine learning


In [161]:
categories_df.to_sql("category1_tbl", con=connection, index=False, if_exists="replace")